In [ ]:
%load_ext autoreload
%autoreload 2
from common import *

In [ ]:
RESULT_JSON = "/Users/law/repos/viper/results/breakdown/breakdown.json"

In [ ]:
from collections import defaultdict
runs = defaultdict(list)

BMS = get_all_runs(RESULT_JSON)

# pprint(BMS)

In [ ]:
INSERT_TIMERS = ("insert-lock", "insert-update", "insert-write")
GET_TIMERS = ("get-fetch", "get-map", "get-read")
UPDATE_TIMERS = ("update-fetch", "update-map", "update-modify") #, "update-write")
DELETE_TIMERS = ("delete-lock", "delete-update", "delete-write")
ALL_TIMERS = (*INSERT_TIMERS, *GET_TIMERS, *UPDATE_TIMERS, *DELETE_TIMERS)

def get_sum_time(bm, timers):
    total_ns = 0
    for timer in timers:
        total_ns += bm[timer]
    return total_ns

st_bm = BMS[0]
at_bm = BMS[1]

st_insert = get_sum_time(BMS[0], INSERT_TIMERS)
st_get = get_sum_time(BMS[0], GET_TIMERS)
st_update = get_sum_time(BMS[0], UPDATE_TIMERS)
st_delete = get_sum_time(BMS[0], DELETE_TIMERS)
at_insert = get_sum_time(BMS[1], INSERT_TIMERS)
at_get = get_sum_time(BMS[1], GET_TIMERS)
at_update = get_sum_time(BMS[1], UPDATE_TIMERS)
at_delete = get_sum_time(BMS[1], DELETE_TIMERS)

bar_width = 0.50
st_pos = np.arange(4)
al_pos = [x + bar_width for x in st_pos]
POS = [st_pos, al_pos]

fig, ax = plt.subplots(1, 1, figsize=(4, 1.5))

STYLES = {
    'pmem': (PMEM_COLOR, ''),
    'map' : (DRAM_COLOR, ''),
    'lock': ("#990000",  ''),
    'val' : ("#009900",  ''),
}

ALL_RUNS = [(st_bm, st_insert, st_get, st_update, st_delete)] #, (at_bm, at_insert, at_get, at_update, at_delete)]
for i, (bm, insert, get, update, delete) in enumerate(ALL_RUNS):
    # INSERT
    insert_fetch = 0
    insert_lock = bm["insert-lock"] / insert
    insert_write = bm["insert-write"] / insert
    insert_update = bm["insert-update"] / insert
    ax.bar(POS[i][0], insert_fetch, bar_width, bottom=0, 
           label="Fetch", color=STYLES['val'][0], hatch=STYLES['val'][1])
    ax.bar(POS[i][0], insert_lock, bar_width, bottom=insert_fetch, 
           label="Lock", color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][0], insert_update, bar_width, bottom=insert_lock + insert_fetch, 
           label="Map", color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][0], insert_write, bar_width, bottom=insert_lock + insert_fetch + insert_update, 
           label="PMem", color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])
        
    # GET
    get_map = bm["get-map"] / get
    get_fetch = bm["get-fetch"] / get
    get_lock = 0.1
    get_read = bm["get-read"] / get
    ax.bar(POS[i][1], get_fetch, bar_width, bottom=0,
           color=STYLES['val'][0], hatch=STYLES['val'][1])
    ax.bar(POS[i][1], get_lock, bar_width, bottom=get_fetch, 
           color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][1], get_map, bar_width, bottom=get_lock + get_fetch, 
           color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][1], get_read, bar_width, bottom=get_map + get_fetch + get_lock,
           color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])
    
    # UPDATE
    update_map = bm["update-map"] / update
    update_fetch = bm["update-fetch"] / update
    update_lock = 0.1
    update_modify = bm["update-modify"] / update
    ax.bar(POS[i][2], update_fetch, bar_width, bottom=0, 
           color=STYLES['val'][0], hatch=STYLES['val'][1])
    ax.bar(POS[i][2], update_lock, bar_width, bottom=update_fetch, 
           color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][2], update_map, bar_width, bottom=update_fetch + update_lock, 
           color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][2], update_modify, bar_width, bottom=update_map + update_fetch + update_lock,
           color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])

        
    # DELETE
    delete_fetch = 0.1
    delete_lock = bm["delete-lock"] / delete
    delete_write = bm["delete-write"] / delete
    delete_update = bm["delete-update"] / delete
    ax.bar(POS[i][3], delete_fetch, bar_width, bottom=0, 
           color=STYLES['val'][0], hatch=STYLES['val'][1])
    ax.bar(POS[i][3], delete_lock, bar_width, bottom=delete_fetch, 
           color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][3], delete_update, bar_width, bottom=delete_fetch + delete_lock, 
           color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][3], delete_write, bar_width, bottom=delete_fetch + delete_lock + delete_update,
           color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])
    
    
plt.rcParams['hatch.linewidth'] = 1.5
plt.rcParams['hatch.color'] = 'white' 


ax.set_xticks([r + (0.0 * bar_width) for r in st_pos])
ax.xaxis.set_tick_params(pad=1)
ax.set_xticklabels(["PUT", "GET", "UPDATE", "DELETE"], fontsize=12)

ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])

ax.set_ylabel("Normalized\nDuration/op", fontsize=12)
ax.yaxis.set_label_coords(-0.13, 0.45)

# Put a legend below current axis
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='center right', bbox_to_anchor=(1.32, 0.45), 
          ncol=1, frameon=False, framealpha=1,
          fontsize=12, columnspacing=0.3, handletextpad=0.2, labelspacing=0.3,
          handlelength=1.8, borderpad=0.1)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(12)

plt.tick_params(axis='x', bottom=False)
ax.set_axisbelow(True)
ax.grid(axis='y', which='major')
hide_border(ax)

fig.savefig('charts/breakdown.pdf', bbox_inches='tight')
fig.savefig('charts/breakdown.svg', bbox_inches='tight')